In [2]:
# load libraries

# %load_ext cudf.pandas
import pandas as pd
import numpy as np
import numba as nb
from numba import cuda
import cupy as cp
import itertools as it

# cuda.detect()

In [3]:
# function used to decorate other functions and calculate the execution time

import time
from functools import wraps

def timer(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"Execution time of {func.__name__}: {end - start} seconds")
        return result
    return wrapper

In [4]:
# this reads csv data files and creates several pandas data frames

@timer
def read_data():

    # if we are in the linux subsystem and need to navigate through linux
    # fdata = "/mnt/c/work/python/draws/data/data_draws.csv"
    # fsegs = "/mnt/c/work/python/draws/data/data_segs.csv"
    # fwts = "/mnt/c/work/python/draws/data/data_weights.csv"

    # in windows
    fdata = "/kaggle/input/david-data/data_draws.csv"
    fsegs = "/kaggle/input/david-data/data_segs.csv"
    fwts = "/kaggle/input/david-data/data_weights.csv"
    
    data = pd.read_csv(fdata)
    segs = pd.read_csv(fsegs)
    wts = pd.read_csv(fwts)
    draw = data['draw']
    
    data_avg = data.groupby('id')[list(data)].agg('mean')
    data_avg['id'] = data_avg['id'].astype(int)
    data_avg['draw'] = 1
    
    # print(list(segs.columns))

    data = data.drop(columns=['id'])
    segs = segs.drop(columns=['id'])
    wts = wts.drop(columns=['id'])
    data_avg = data_avg.drop(columns=['id'])
    
    # print(data.shape)
    # print(segs.shape)
    # print(wts.shape)

    # this is a dummy list/array used for testing. In the optimize function, this is created properly.
    lvector = [[0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 3.99],
       [0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 3.99],
       [0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 4.99],
       [0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 5.99]]

    xvector = cp.ascontiguousarray(cp.array(lvector, cp.float64))
    
    return(data, segs, wts, draw, data_avg, xvector.T)

In [5]:
# this is the main calculation function

def gpu_calculate_prob(x, parms, seg_wt_t, seg_wt_sums):
    
    util = cp.dot(parms, x)
    eutil = cp.exp(util)
    sume = cp.sum(eutil, axis=1)
    sume = cp.reshape(sume, (sume.size, 1))
    prob = cp.divide(eutil, sume)
    prob_seg_sum = cp.dot(seg_wt_t, prob)
    prob_seg = cp.divide(prob_seg_sum, seg_wt_sums)
    
    return(prob_seg)

In [6]:
# this function sets up variables in the proper format
@timer
def setup_vars(d, s, w):
    # x = get_xmatrix(use_cuda = True)

    vdata = cp.asarray(d.values)
    vsegs = cp.asarray(s.values)
    vwts = cp.asarray(w.values)
    vwts = cp.reshape(vwts, (vwts.size, 1))

    # the largest draw values should equal the number of draws
    num_draws = max(d["draw"])
    
    # number of rows
    num_resp = s.shape[0]

    # np.concatenate does not work in an nb.njit function
    seg_wt = cp.concatenate([vsegs * vwts], axis=1)

    seg_wt_sums = cp.sum(seg_wt, axis=0)

    seg_wt_sums = cp.reshape(seg_wt_sums, (seg_wt_sums.size, 1))

    seg_wt_t =  seg_wt.T
    
    # print("vdata", vdata.device)
    # print("vsegs", vsegs.device)
    # print("vwts", vwts.device)
    # print("seg_wt", seg_wt.device)
    # print("seg_wt_sums", seg_wt_sums.device)
    # print("seg_wt_t", seg_wt_t.device)
    
    return(num_draws, num_resp, vdata, seg_wt_t, seg_wt_sums)
    

In [7]:
# This code is an example of the draws implementation
# It introduces the draws calculation loop (for i in range(num_draws):). This loop should be executed in parallel. 

def draws_run(num_draws, num_resp, x, vdata, seg_wt_t, seg_wt_sums):

    prob_lst = [0 for i in range(num_draws)]
    
    # print("x", x.device)
    # print("vdata", vdata.device)
    # print("seg_wt_t", seg_wt_t.device)
    # print("seg_wt_sums", seg_wt_sums.device)
    

    for i in range(num_draws):

        # if i counts starting at 0
        slice_1 = (i) * num_resp
        slice_2 = (i+1) * num_resp

        ncols = vdata.shape[1] -1 # drop the "draw" column, assumes it is in the last position. Could be more explicit here. This line could cause errors.
        parms = cp.ascontiguousarray(vdata[slice_1:slice_2, 0:ncols])
        prob_lst[i] = gpu_calculate_prob(x, parms, seg_wt_t, seg_wt_sums)


    # combine the list into an array
    prob_stack = cp.stack(prob_lst)

    # calculate mean, stdev, and confidence intervals
    prob_mean = cp.mean(prob_stack, axis=0)
    prob_stdev = cp.std(prob_stack, axis=0)
    prob_ci = prob_stdev * 1.96 / cp.sqrt(num_draws)

    # return results
    results = [prob_mean, prob_ci]
    
    return(results)


In [8]:
# this code adds a timer to the draws_run
# this was added because if we call draws_run() from the optimization, we do not want to print the execution time for each iteration

@timer
def draws_run_timer(num_draws, num_resp, x, vdata, seg_wt_t, seg_wt_sums):
    return draws_run (num_draws, num_resp, x, vdata, seg_wt_t, seg_wt_sums)
                      

In [9]:
# implement and test the draws run

def implement_draws_run():
    # get the data
    d, s, w, draw, d_avg, x = read_data()


    # set up the variables
    # if we pass d, we will get back a large file with 200 draws
    # if we pass d_avg, we will get back a small file with 1 draw
    num_draws, num_resp, vdata, seg_wt_t, seg_wt_sums = setup_vars(d, s, w)

    # print("num_draws", num_draws)
    # print("num_resp", num_resp)
    # print("x.shape", x.shape)
    # print("vdata.shape", vdata.shape)

    # run the draws function
    return draws_run_timer(num_draws, num_resp, x, vdata, seg_wt_t, seg_wt_sums)

# print("prob_mean", prob_mean)    
# print("prob_ci", prob_ci)
# print(len(res_list))

temp = implement_draws_run()



Execution time of read_data: 5.738913536071777 seconds
Execution time of setup_vars: 0.40834689140319824 seconds
Execution time of draws_run_timer: 2.35496187210083 seconds


In [10]:
# a function to get a matrix of dummy or effects codes
# this is used to build the x-matrix

def get_coding(num_levels, is_dummy, is_gpu):
    # num_levels must be > 1
    
    if is_gpu == True:
        if is_dummy == False:
            x = cp.identity(num_levels-1)
            newrow = cp.ones(num_levels-1)*-1
            x = cp.vstack([x, newrow])
        else:
            x = cp.identity(num_levels)
    else:
        if is_dummy == False:
            x = np.identity(num_levels-1)
            newrow = np.ones(num_levels-1)*-1
            x = np.vstack([x, newrow])            
        else:
            x = np.identity(num_levels)
        
    # it is better to return the list than the array because we need to append to this later
    return(x.tolist())



# # testing code
# x = get_coding(2, True, True)
# print(type(x))
# print(x)

# x = get_coding(2, False, True)
# print(type(x))
# print(x)

In [11]:
# this will need to be customized for each problem & data file
@timer
def setup_the_problem():
    
    # get the data
    d, s, w, draw, d_avg, x = read_data()
    
    # set up the variables
    num_draws, num_resp, vdata, seg_wt_t, seg_wt_sums = setup_vars(d_avg, s, w)

    # set up the coding for the x-matrix
    is_dummy = True
    is_gpu = True

    att_0 = get_coding(7, is_dummy, is_gpu)
    att_1 = get_coding(4, is_dummy, is_gpu)
    att_2 = get_coding(4, is_dummy, is_gpu)
    att_3 = get_coding(4, is_dummy, is_gpu)
    att_4 = get_coding(7, is_dummy, is_gpu)
    att_5 = get_coding(4, is_dummy, is_gpu)
    att_6 = get_coding(4, is_dummy, is_gpu)
    att_7 = get_coding(7, is_dummy, is_gpu)
    att_8 = get_coding(4, is_dummy, is_gpu)
    att_9 =  [[2.99], [3.99]]

    # set up lists of the valid levels to test
    a_0  = np.arange(7).tolist()
    a_1  = np.arange(4).tolist()
    a_2  = np.arange(4).tolist()
    a_3  = np.arange(4).tolist()
    a_4  = np.arange(7).tolist()
    a_5  = np.arange(4).tolist()
    a_6  = np.arange(4).tolist()
    a_7  = np.arange(7).tolist()
    a_8  = np.arange(4).tolist()
    a_9 = np.arange(2).tolist()

    # combine the list of levels into an attributes list
    atts = [a_0, a_1, a_2, a_3, a_4, a_5, a_6, a_7, a_8, a_9]
    
    # combine the list of coded levels into a coded attributes list
    atts_code = [att_0, att_1, att_2, att_3, att_4, att_5, att_6, att_7, att_8, att_9]

    # define the fixed alternatives
    # alt_2 - alt_4 are fixed. We only want to calculate the combinations of alt_1
    alt_2 = [0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 3.99]
    alt_3 = [0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 4.99]
    alt_4 = [0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 5.99]

    # combine the fixed alternatives
    alts_fixed = [alt_2, alt_3, alt_4]
    
    return atts, atts_code, alts_fixed, num_draws, num_resp, vdata, seg_wt_t, seg_wt_sums


In [17]:
# this is the main optimization routine
# this code should be consistent across projects
# I would like to see the combination loop executed in parallel (for c in combination:)
@timer
def optimize(test_number_of_combos, atts, atts_code, alts_fixed, num_draws, num_resp, vdata, seg_wt_t, seg_wt_sums):
   
    combination = [p for p in it.product(*atts)]
    # here's a line for testing a small number of combinations
    if test_number_of_combos > 0:
        combination = combination[:test_number_of_combos]

    res_lst = []
    for c in combination:
        
        # combine the attributes into an alternative
        # alt_1 = att_0[c[0]] + att_1[c[1]] + att_2[c[2]] + att_3[c[3]] + att_4[c[4]] + att_5[c[5]] + att_6[c[6]] + att_7[c[7]] + att_8[c[8]] + att_9[c[9]]    
        for i in range(len(atts)):
            # print(atts_code[i][c[i]])
            if i == 0:
                alt_1 = atts_code[i][c[i]]
            else:
                alt_1 = alt_1 + atts_code[i][c[i]]
        
        # combine the new alternative with the fixed alternatives
        alts = []
        for i in range(len(alts_fixed)+1):
            if i == 0:
                alts.append(alt_1)
            else:
                alts.append(alts_fixed[i-1])
                
        # convert the alternatives to a transposed x-matrix
        x = cp.array(alts, cp.float64).T
        
        
        # this code block is the same as the draws_run() function, and implementation with the draws_run() function is below
        prob_lst = [0 for i in range(num_draws)]

        for i in range(num_draws):

            # if i counts starting at 0
            slice_1 = (i) * num_resp
            slice_2 = (i+1) * num_resp

            ncols = vdata.shape[1] -1 # drop the "draw" column
            parms = cp.ascontiguousarray(vdata[slice_1:slice_2, 0:ncols])
            prob_lst[i] = gpu_calculate_prob(x, parms, seg_wt_t, seg_wt_sums)

        prob_stack = cp.stack(prob_lst)
        prob_mean = cp.mean(prob_stack, axis=0)
        prob_stdev = cp.std(prob_stack, axis=0)
        prob_ci = prob_stdev * 1.96 / cp.sqrt(num_draws)
        results = [prob_mean, prob_ci]

        # res_lst.append(results)
        
    return(results)


In [18]:
# this is an implementation of the optimization problem
def implement_optimization_run():
    # set up the problem variables
    atts, atts_code, alts_fixed, num_draws, num_resp, vdata, seg_wt_t, seg_wt_sums = setup_the_problem()

    # limit the runtime by running only a subset of combinations. 
    # if test_number_of_combos = 0, then it will run all 2.8 million combinations
    # if test_number_of_combos = x, then it will only run the "x" combinations

    test_number_of_combos = 10_000
    # test_number_of_combos = 1000
    # test_number_of_combos = 0

    # run the optimization
    return optimize(test_number_of_combos, atts, atts_code, alts_fixed, num_draws, num_resp, vdata, seg_wt_t, seg_wt_sums)

temp = implement_optimization_run()

Execution time of read_data: 5.494206190109253 seconds
Execution time of setup_vars: 0.002137899398803711 seconds
Execution time of setup_the_problem: 5.505829095840454 seconds
Execution time of optimize: 8.87936806678772 seconds


In [19]:
atts, atts_code, alts_fixed, num_draws, num_resp, vdata, seg_wt_t, seg_wt_sums = setup_the_problem()

Execution time of read_data: 5.391966819763184 seconds
Execution time of setup_vars: 0.0022640228271484375 seconds
Execution time of setup_the_problem: 5.406954288482666 seconds


In [20]:
test_number_of_combos = 10_000

In [21]:
combination = [p for p in it.product(*atts)]
# here's a line for testing a small number of combinations
if test_number_of_combos > 0:
    combination = combination[:test_number_of_combos]

In [23]:
res_lst = []

In [24]:
c = combination[0]

In [37]:
for i in range(len(atts)):
    # print(atts_code[i][c[i]])
    if i == 0:
        alt_1 = atts_code[i][c[i]]
    else:
        alt_1 = alt_1 + atts_code[i][c[i]]

In [30]:
i = 0

In [53]:
len(atts)

10

In [36]:
c

(0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

In [33]:
combination[100]

(0, 0, 0, 0, 0, 0, 1, 5, 2, 0)

In [28]:
atts

[[0, 1, 2, 3, 4, 5, 6],
 [0, 1, 2, 3],
 [0, 1, 2, 3],
 [0, 1, 2, 3],
 [0, 1, 2, 3, 4, 5, 6],
 [0, 1, 2, 3],
 [0, 1, 2, 3],
 [0, 1, 2, 3, 4, 5, 6],
 [0, 1, 2, 3],
 [0, 1]]

In [29]:
atts_code

[[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]],
 [[1.0, 0.0, 0.0, 0.0],
  [0.0, 1.0, 0.0, 0.0],
  [0.0, 0.0, 1.0, 0.0],
  [0.0, 0.0, 0.0, 1.0]],
 [[1.0, 0.0, 0.0, 0.0],
  [0.0, 1.0, 0.0, 0.0],
  [0.0, 0.0, 1.0, 0.0],
  [0.0, 0.0, 0.0, 1.0]],
 [[1.0, 0.0, 0.0, 0.0],
  [0.0, 1.0, 0.0, 0.0],
  [0.0, 0.0, 1.0, 0.0],
  [0.0, 0.0, 0.0, 1.0]],
 [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]],
 [[1.0, 0.0, 0.0, 0.0],
  [0.0, 1.0, 0.0, 0.0],
  [0.0, 0.0, 1.0, 0.0],
  [0.0, 0.0, 0.0, 1.0]],
 [[1.0, 0.0, 0.0, 0.0],
  [0.0, 1.0, 0.0, 0.0],
  [0.0, 0.0, 1.0

In [14]:
# this is the optimization run using the draws_run() function 
# it is exactly the same as the function above, but i replaced some code with the draws_run() function1

# I would like to see the combination loop executed in parallel (for c in combination:)
@timer
def optimize_draws(test_number_of_combos, atts, atts_code, alts_fixed, 
                   num_draws, num_resp, vdata, seg_wt_t, seg_wt_sums):
   
    
    combination = [p for p in it.product(*atts)]
    # here's a line for testing a small number of combinations
    if test_number_of_combos > 0:
        combination = combination[:test_number_of_combos]

    # print(len(combination))
    
    # print(type(test_number_of_combos))
    # print(type(atts))
    # print(type(atts_code))
    # print(type(alts_fixed))
    # print(type(combination))

    # print("combination", combination.device)

    res_lst = []
    for c in combination:
        
        # combine the attributes into an alternative
        # alt_1 = att_0[c[0]] + att_1[c[1]] + att_2[c[2]] + att_3[c[3]] + att_4[c[4]] + att_5[c[5]] + att_6[c[6]] + att_7[c[7]] + att_8[c[8]] + att_9[c[9]]    
        for i in range(len(atts)):
            # print(atts_code[i][c[i]])
            if i == 0:
                alt_1 = atts_code[i][c[i]]
            else:
                alt_1 = alt_1 + atts_code[i][c[i]]
        
        # combine the new alternative with the fixed alternatives
        alts = []
        for i in range(len(alts_fixed)+1):
            if i == 0:
                alts.append(alt_1)
            else:
                alts.append(alts_fixed[i-1])
                
        # convert the alternatives to a transposed x-matrix
        x = cp.array(alts, cp.float64).T
        
        # run the draws function
        results = draws_run(num_draws, num_resp, x, vdata, seg_wt_t, seg_wt_sums)

        res_lst.append(results)
        
    return(res_lst)


In [15]:
# this is an implementation of the optimization problem

def implement_optimization_draws_run():
    # set up the problem variables
    atts, atts_code, alts_fixed, num_draws, num_resp, vdata, seg_wt_t, seg_wt_sums= setup_the_problem()

    # limit the runtime by running only a subset of combinations. 
    # if test_number_of_combos = 0, then it will run all 2.8 million combinations
    # if test_number_of_combos = x, then it will only run the "x" combinations

    test_number_of_combos = 10_000
    # test_number_of_combos = 1000
    # test_number_of_combos = 0

    # run the optimization
    return optimize_draws(test_number_of_combos, atts, atts_code, alts_fixed, num_draws, num_resp, vdata, seg_wt_t, seg_wt_sums)

temp = implement_optimization_draws_run()



Execution time of read_data: 5.479531526565552 seconds
Execution time of setup_vars: 0.0019359588623046875 seconds
Execution time of setup_the_problem: 5.491311311721802 seconds
Execution time of optimize_draws: 9.58283805847168 seconds
